# High-level LSTM Keras (CNTK) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from common.params_lstm import *
from common.utils import *

Using CNTK backend


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print(K.backend.backend())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.0.8
Numpy:  1.13.1
CNTK:  2.1
cntk


In [3]:
def create_symbol():
    model = Sequential()
    model.add(Embedding(MAXFEATURES, NUMHIDDEN, input_length=MAXLEN))
    model.add(LSTM(NUMHIDDEN, recurrent_dropout=DROPOUT))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [4]:
def init_model(m):
    m.compile(
        loss = "binary_crossentropy",
        optimizer = K.optimizers.Adam(LR, BETA_1, BETA_2, EPS),
        metrics = ['accuracy'])
    return m

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Trimming to 25000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int32 int32 int32 int32
CPU times: user 5.94 s, sys: 364 ms, total: 6.31 s
Wall time: 7.61 s


In [6]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 272 ms, sys: 624 ms, total: 896 ms
Wall time: 624 ms


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 12 ms, sys: 52 ms, total: 64 ms
Wall time: 10.4 ms


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          2500000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 2,580,501
Trainable params: 2,580,501
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/3
25000/25000 [==============================] - 77s - loss: 0.5498 - acc: 0.7314    - ETA: 0s - loss: 0.5500 - acc: 0.73
Epoch 2/3
25000/25000 [==============================] - 77s - loss: 0.4672 - acc: 0.7965    - ETA
Epoch 3/3
25000/25000 [==============================] - 78s - loss: 0.3830 - acc: 0.8451    - ETA: 1s - loss: 0.3837 - acc
CPU times: user 7min 19s, sys: 27.5 s, total: 7min 46s
Wall time: 3min 53s


In [10]:
%%time
y_guess = (model.predict(x_test, batch_size=BATCHSIZE))>=0.5

CPU times: user 1min 7s, sys: 4.31 s, total: 1min 11s
Wall time: 36.1 s


In [11]:
print("Accuracy: ", sum(y_guess.squeeze() == y_test)/len(y_guess))

Accuracy:  0.83336
